In [4]:
import os
import pandas as pd
from functions import get_all_origins, find_pattern_for_quantity, convert_to_grams, relation_qnt_preco, remove_spaces, clean_text
import re
import numpy as np
import ast

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = get_all_origins('back_words_contained_in_the_title')
df.head()




,ref,back_word,location,subject,row_number
0,3fc276b6,real,36,creatina,1
1,3fc276b6,para,36,creatina,2
2,3fc276b6,ganhos,36,creatina,3
3,3fc276b6,reais,36,creatina,4
4,3fc276b6,a,36,creatina,5
